In [27]:
import pandas as pd
from neo4j import GraphDatabase
import csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
import ast


In [65]:
df = pd.read_csv("final_movie.csv")

df.head(5)

df = df[(df['genres'].str.strip() != "[]") & (df['Director'].str.strip() != "[]")]



In [72]:
df

,keywords,cast,crew,Director,genres,imdb_id,original_language,revenue,release_date,spoken_languages,title,vote_average,vote_count,overview
0,jealousy toy boy friendship friends rivalry bo...,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",['John Lasseter'],"['Animation', 'Comedy', 'Family']",tt0114709,en,373554033.0,1995,"[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ..."
1,boardgame disappearance basedonchildren'sbook ...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",['Joe Johnston'],"['Adventure', 'Fantasy', 'Family']",tt0113497,en,262797249.0,1995,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji,6.9,2413.0,When siblings Judy and Peter discover an encha...
2,fishing bestfriend duringcreditsstinger oldmen,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",['Howard Deutch'],"['Romance', 'Comedy']",tt0113228,en,0.0,1995,"[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men,6.5,92.0,A family wedding reignites the ancient feud be...
3,basedonnovel interracialrelationship singlemot...,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",['Forest Whitaker'],"['Comedy', 'Drama', 'Romance']",tt0114885,en,81452156.0,1995,"[{'iso_639_1': 'en', 'name': 'English'}]",Waiting to Exhale,6.1,34.0,"Cheated on, mistreated and stepped on, the wom..."
4,baby midlifecrisis confidence aging daughter m...,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",['Charles Shyer'],['Comedy'],tt0113041,en,76578911.0,1995,"[{'iso_639_1': 'en', 'name': 'English'}]",Father of the Bride Part II,5.7,173.0,Just when George Banks has recovered from his ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46621,NaN,"[{'cast_id': 1, 'character': 'Kira (as Cassand...","[{'credit_id': '5757f36ac3a3687d6f000e8a', 'de...",['Aaron Osborne'],['Science Fiction'],tt0112613,en,0.0,1995,"[{'iso_639_1': 'en', 'name': 'English'}]",Caged Heat 3000,3.5,1.0,It's the year 3000 AD. The world's most danger...
46622,NaN,"[{'cast_id': 1, 'character': 'Sir Robert Hode'...","[{'credit_id': '52fe44439251416c9100a899', 'de...",['John Irvin'],"['Drama', 'Action', 'Romance']",tt0102797,en,0.0,1991,"[{'iso_639_1': 'en', 'name': 'English'}]",Robin Hood,5.7,26.0,"Yet another version of the classic epic, with ..."
46623,tragiclove,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",['Hamid Nematollah'],"['Drama', 'Family']",tt6209470,fa,0.0,NaT,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Subdue,4.0,1.0,Rising and falling between a man and woman.
46624,artist play pinoy,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",['Lav Diaz'],['Drama'],tt2028550,tl,0.0,2011,"[{'iso_639_1': 'tl', 'name': ''}]",Century of Birthing,9.0,3.0,An artist struggles to finish his work while a...


In [67]:
# Sampling data
df_movies = df.copy()
sub_df = df_movies.sample(n=1000, random_state=42)


In [70]:
# get unique genres
genres = []
sub_df['genres'].apply(lambda row: genres.extend(ast.literal_eval(row)))

# get unique directors
directors = []
sub_df['Director'].apply(lambda row: directors.extend(ast.literal_eval(row)))


genres = set(genres)
directors = set(directors)

In [71]:
directors

{'SABU',
 'John S. Robertson',
 'Akan Satayev',
 'Jerzy Hoffman',
 'Lambert Hillyer',
 'Len Wiseman',
 'Denis Neimand',
 'Jacqui Morris',
 'Jonathan Nossiter',
 'François Ozon',
 'Terry Marcel',
 'David Weaver',
 'Jordan Vogt-Roberts',
 'Hiroyuki Yamashita',
 'Alan J. Pakula',
 'Makoto Shinkai',
 'D.A. Pennebaker',
 'Alexander Hall',
 'Rick Jacobson',
 'Roy Del Ruth',
 'Steve Antin',
 'Scott McGehee',
 'Jordan Brady',
 'Craig Gillespie',
 'Sylvain White',
 'Allan Burns',
 'Juris Poškus',
 'Phyllis Ferguson',
 'Arthur Lubin',
 'James Foley',
 'André De Toth',
 'Nate Meyer',
 'Luchino Visconti',
 'Roel Reiné',
 'Malcolm Mowbray',
 'Martial Fougeron',
 'Sreenu Vaitla',
 'Billy Wilder',
 'Jake Newsome',
 'Christopher Hampton',
 'Thilo Rothkirch',
 'Daisuke Tengan',
 'Jan Švankmajer',
 'Lloyd Richards',
 'Andrew Lau',
 'Morgan J. Freeman',
 'Mark Robson',
 'Thomas G. Miller',
 'Sidney Lumet',
 'José Ferrer',
 'Shoaib Mansoor',
 'Wilfred Jackson',
 'Alan Taylor',
 'Lech Majewski',
 'Martin R

## Setting up graph database

In [10]:
uri = "neo4j://localhost:7687"
user = "neo4j"
password = "vampire-float-olivia-maestro-sleep-4222" 

#### Make sure you can run this on your browser - initiate a Neo4j server ####

In [11]:
driver = GraphDatabase.driver(uri, auth=(user, password))

In [43]:
# kill switch
def delete_all_data(tx):
    tx.run("MATCH (n) DETACH DELETE n")

with driver.session() as session:
    session.write_transaction(delete_all_data)

driver.close()

/var/folders/5w/zg23p5bd3bnc86d75_pkn99m0000gn/T/ipykernel_98724/3803800313.py:6: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(delete_all_data)


### Movie node

In [44]:
def create_movie_node(tx, row):
    tx.run("""
    CREATE (:movie {
            title: $title,
            overview: $overview,
            vote_average: $vote_average, 
            vote_count: $vote_count

    })
    """, title=row['title'], overview=row['overview'], vote_average=row['vote_average'],
                vote_count=row['vote_count'])
    

with driver.session() as session:
    for index, row in sub_df.iterrows():
        session.write_transaction(create_movie_node, row)

    

/var/folders/5w/zg23p5bd3bnc86d75_pkn99m0000gn/T/ipykernel_98724/556782445.py:16: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_movie_node, row)


### Director Node

In [ ]:
def create_director_node(tx, row):
    tx.run("""
    
    
    CREATE (:director {
            name
    })
    
    
    """)